# Tarefa
__________
Resolver o problema do rato utilizando Q-Table. Você deve programar o ambiente e o algoritmo de Reinforcement Learning.

![Rato](https://i.imgur.com/rYnjoet.png)

Sobre o ambiente:
  * O episódio deve terminar quando o rato alcançar a pilha de queijos ou tomar o veneno.
  * O objetivo é fazer com que o rato pegue todos os queijos do mapa sem tomar o veneno.
  * As ações devem ser mover o rato, em 1 casa, para cima, baixo, esquerda e direita.
  * O rato está confinado no espaço de 6 casas, conforme a imagem abaixo.

Dicas:
  * É possível completar toda a tarefa utilizando apenas numpy.
  * Ler sobre Q-Learning na referência \[1\].
  * Representar o mapa como uma matriz.

Tabela Q sugerida:

![Q-Table](https://i.imgur.com/MWWvHZl.png)

Referências:

1. PALANISAMY, Praveen. Hands-On Intelligent Agents with OpenAI Gym: Your guide to   developing AI agents using deep reinforcement learning. Packt Publishing Ltd, 2018.

# Entrega
___________

Prazo: **26/02**

Enviar para **lbpires@latam.stefanini.com**

# Código
____

### Parâmetros

In [0]:
class TrainingParameters:
    def __init__(self, max_episodes: int, steps_per_episode: int):  
        self.max_episodes = int(max_episodes)
        self.steps_per_episode = int(steps_per_episode)


class AgentParameters:
    def __init__(self, epsilon_min, epsilon_decay, start_epsilon):
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.start_epsilon = start_epsilon


class LearningParameters:
    def __init__(self, alpha, gamma):
        self.alpha = alpha
        self.gamma = gamma

### Q-Table

In [0]:
import numpy as np

class QTable:
    def __init__(self, num_states: int, num_actions: int, params: LearningParameters):
        self.table = np.random.rand(num_states, num_actions)
        self.gamma = params.gamma
        self.alpha = params.alpha

    # calculates Q-table values
    def learn(self, obs, action_index, reward, next_obs):
        deltaQ = reward + self.gamma*np.max(self.table[next_obs]) - self.table[obs, action_index]
        self.table[obs, action_index] = self.table[obs,action_index] + self.alpha*deltaQ

### Agente

In [0]:
class Agent:
    def __init__(self, params: AgentParameters, actions_shape):
        self.epsilon_min = params.epsilon_min
        self.epsilon_decay = params.epsilon_decay
        self.epsilon = params.start_epsilon
        self.actions = [i for i in range(actions_shape)]


    def set_q_table(self, q_table: QTable):
        self.q_table = q_table


    def set_actions(self, actions):
        self.actions = actions


    def get_action(self, obs):
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay
        if np.random.random() > self.epsilon:
            action_index = np.argmax(self.q_table.table[obs,:])
            self.action = self.actions[action_index]            
        else:
            action = np.random.choice(self.actions)
            self.action = action
        return self.action                
        

    def set_action(self, action_index: int):
        self.action_index = action_index
        self.action = self.actions[action_index]
        

### Ambiente

Para a criação do ambiente, teve-se que pensar nos possíveis estados dele. Mesmo que o campo inteiro seja composto por 6 possíveis lugares para o rato estar, como existem campos com queijos e que não encerram o episódio, deve-se levar em conta que o rato na mesma posição pode significar mais de um estado, como na imagem a seguir.

![Rato](https://i.imgur.com/JDwfYqs.png)
![Rato](https://i.imgur.com/5OEUOfA.png)

![Rato](https://i.imgur.com/xsp4MY0.png)
![Rato](https://i.imgur.com/dLAKfuH.png)


Portanto, para esse exemplo, existem 4 possíveis estados com o rato na mesma posição, sendo que seu estado depende diretamente da existência ou não de queijos em seus respectivos espaços. Levou-se em consideração todos os possíveis estados em cada posição do rato. O estado em si é ou obtido diretamente pela leitura da matriz ou pela execução do método indicado que tanto leva em consideração a existência ou não de queijos em posições específicas quanto um *range* que não permite a sobreposição de número de estados.

A matriz de estados, métodos e controle dos estados foram feitos no bloco de código a seguir.



In [0]:
class RatEnv:
    def __init__(self):
        self.grid = [['R', 5, 0],[100, -1000, 1000]]
        self.row = 0
        self.column = 0
        self.states = [[self.get_0_0, self.get_0_1, self.get_0_2],[self.get_1_0, 12, 13]]
    
    def reset(self):
        self.grid = [['R', 5, 0],[100, -1000, 1000]]
        self.row = 0
        self.column = 0
        self.states = [[self.get_0_0, self.get_0_1, self.get_0_2],[self.get_1_0, 10, 11]]
        return 0

    # Métodos responsáveis por obter o valor do estado baseando-se na existência ou não de 
    # queijos nos lugares pré-determinados.

    # Método que indica o estado quando o rato esta na posição (0,0)
    def get_0_0(self):
        return bool(self.grid[0][1]) + bool(self.grid[1][0])

    # Método que indica o estado quando o rato esta na posição (0,1)
    def get_0_1(self):
        return bool(self.grid[1][0]) + 6

    # Método que indica o estado quando o rato esta na posição (0,2)
    def get_0_2(self):
        return bool(self.grid[1][0]) + 8

    # Método que indica o estado quando o rato esta na posição (1,0)
    def get_1_0(self):
        return bool(self.grid[0][1]) + 4

    # Método que aplica uma ação, atualiza o ambiente e retorna o estado
    def step(self, action):
        done = False
        reward = 0
        if (action == 'RIGHT' or action == 3):
            if (self.column + 1 >= len(self.grid[0])):
                reward = 0
            else:
                reward = self.grid[self.row][self.column + 1]
                self.grid[self.row][self.column] = 0
                self.grid[self.row][self.column + 1] = 'R'
                self.column += 1
                if (abs(reward) > 500):
                    done = True
        elif (action == 'LEFT' or action == 1):
            if (self.column - 1 < 0):
                reward = 0
            else:
                reward = self.grid[self.row][self.column - 1]
                self.grid[self.row][self.column] = 0
                self.grid[self.row][self.column - 1] = 'R'
                self.column -= 1
                if (abs(reward) > 500):
                    done = True

        elif (action == 'UP' or action == 2):
            if (self.row - 1 < 0):
                reward = 0
            else:
                reward = self.grid[self.row - 1][self.column]
                self.grid[self.row][self.column] = 0
                self.grid[self.row - 1][self.column] = 'R'
                self.row -= 1
                if (abs(reward) > 500):
                    done = True
        elif (action == 'DOWN' or action == 0):
            if (self.row + 1 >= len(self.grid)):
                reward = 0
            else:
                reward = self.grid[self.row + 1][self.column]
                self.grid[self.row][self.column] = 0
                self.grid[self.row + 1][self.column] = 'R'
                self.row += 1
                if (abs(reward) > 500):
                    done = True

        # Obtenção do estado a partir da matriz de estados.
        # Caso o estado seja um método, ele deve ser executado pois seu valor depende
        # da existência ou não de ao menos um queijo do ambiente
        state = self.states[self.row][self.column]
        if (not isinstance(state, int)):
            state = state()
        return state, reward - 1, done, "debug stuff here"

    def render(self):
        for r in self.grid:
            print(r)

In [0]:
# Parâmetros para o modelo

EPSILON_MIN = 0.05
MAX_NUM_EPISODES = 1000
STEPS_PER_EPISODE = 10
max_num_steps = MAX_NUM_EPISODES * STEPS_PER_EPISODE
ALPHA = 0.01
GAMMA = 0.999
EPSILON_DECAY = 25 * EPSILON_MIN / max_num_steps
np.random.seed(123)


# Objetos utilizados no sistema com os parâmetros especificados e calculados acima
learning_parameters = LearningParameters(ALPHA, GAMMA)
agent_parameters = AgentParameters(EPSILON_MIN, EPSILON_DECAY, 1)
training_parameters = TrainingParameters(MAX_NUM_EPISODES, STEPS_PER_EPISODE)

# Criação da tabela Q com 12 estados
rat_q_table = QTable(12, 4, learning_parameters)

# Criação do agente e indicação de sua tabela Q
rat_agent = Agent(agent_parameters, 4)
rat_agent.set_q_table(rat_q_table)

# Criação do ambiente
env = RatEnv()

### Treino

In [0]:
# Função responsável por treinar o agente a partir dos rewards e com uma limitação
# número máximo de passos possíveis por episódio
def train(agent: Agent, env, params: TrainingParameters):
    best_reward = -float('inf')
    for episode in range(MAX_NUM_EPISODES):
        obs = env.reset()
        done = False
        total_reward = 0.0
        steps = 0
        while (not done and steps < params.steps_per_episode):
            steps += 1
            action = agent.get_action(obs)
            next_obs, reward, done, info = env.step(action)
            agent.q_table.learn(obs, action, reward, next_obs)
            obs = next_obs
            total_reward += reward
            # if total_reward < 0:
            #     break
        if total_reward > best_reward:
            best_reward = total_reward
        print("Episode#:{} reward:{} best_reward:{} eps:{}".format(episode,
            total_reward, best_reward, agent.epsilon))
    return np.argmax(agent.q_table.table, axis=1)

In [0]:
# Treino e obtenção da política de ação por estado
learned_policy = train(rat_agent, env, training_parameters)

Episode#:0 reward:95.0 best_reward:95.0 eps:0.9987499999999996
Episode#:1 reward:95.0 best_reward:95.0 eps:0.9974999999999992
Episode#:2 reward:-908.0 best_reward:95.0 eps:0.9964999999999988
Episode#:3 reward:95.0 best_reward:95.0 eps:0.9952499999999984
Episode#:4 reward:-902.0 best_reward:95.0 eps:0.9949999999999983
Episode#:5 reward:-5.0 best_reward:95.0 eps:0.9937499999999979
Episode#:6 reward:-5.0 best_reward:95.0 eps:0.9924999999999975
Episode#:7 reward:-997.0 best_reward:95.0 eps:0.9922499999999974
Episode#:8 reward:95.0 best_reward:95.0 eps:0.990999999999997
Episode#:9 reward:-5.0 best_reward:95.0 eps:0.9897499999999966
Episode#:10 reward:-902.0 best_reward:95.0 eps:0.9894999999999965
Episode#:11 reward:-997.0 best_reward:95.0 eps:0.9892499999999964
Episode#:12 reward:998.0 best_reward:998.0 eps:0.9883749999999961
Episode#:13 reward:-1002.0 best_reward:998.0 eps:0.9874999999999958
Episode#:14 reward:-904.0 best_reward:998.0 eps:0.9869999999999957
Episode#:15 reward:95.0 best_rew

In [0]:
learned_policy

array([0, 3, 0, 3, 2, 2, 3, 3, 0, 0, 0, 3])

### Teste

In [0]:
# Função responsável por testar a política aprendida a partir da realização da ação
# de acordo com o estado atual e verificação do ambiente, do estado e da pontuação
# geral obtida
def test(agent: Agent, env, policy):
    done = False
    obs = env.reset()
    total_reward = 0.0
    steps = 0
    actions = ['DOWN', 'LEFT', 'UP', 'RIGHT']
    env.render() 
    while not done:
        action = policy[obs]
        print(actions[action])
        next_obs, reward, done, info = env.step(action)   
        env.render()     
        obs = next_obs
        total_reward += reward
        # print('total_reward: ', total_reward)
        steps += 1
        if steps > 10:
            break
    return total_reward

In [0]:
# Teste a partir da política obtida
total = test(rat_agent, env, learned_policy)

print('TOTAL>>>>>>>', total)

['R', 5, 0]
[100, -1000, 1000]
DOWN
[0, 5, 0]
['R', -1000, 1000]
UP
['R', 5, 0]
[0, -1000, 1000]
RIGHT
[0, 'R', 0]
[0, -1000, 1000]
RIGHT
[0, 0, 'R']
[0, -1000, 1000]
DOWN
[0, 0, 0]
[0, -1000, 'R']
TOTAL>>>>>>> 1100.0


A partir do output acima, verificamos que o caminho seguido pelo rato foi DOWN-UP-RIGHT-RIGHT-DOWN, que representa o caminho em que ele pega todos os queijos.